## Questions

#### Ane
Bruker du TranformerLens biblioteket?


#### Theory (til GPT/research)
Hvordan velges tokens, og hvordan velger modellen alltid den "lengste mulige" token (hvis f.eks. ' cat' og ' ', 'c', 'a', 't' er tokens)?




## Glossary
- Patching: Patching means to switch activations from one run of a model (input A) in certain parts of the neural network with the activations from another run of the model (input B) in certain specified locations in the network. 
- Causal/autoregressive: Uses only previous tokens for prediction.
- Logits: (Unsoftmaxed) output vector (length d_vocab) after unembedding.
- Transformer config:
    * batch: input index in batch.
    * position: token position.
    * d_model: residual stream/embedding dimension
    * d_vocab: vocab size
    * n_ctx: context size
    * d_head: dimension of key/query space in attention head
    * d_mlp: mpl hidden layer size (4 x d_model)
    * n_heads: number of attention heads (d_model/d_head)
    * n_layers: number of layers
- Transformer matrices:
    * $W_E$: Embedding matrix
    * $W_U$: Unembedding matrix
- Residual stream: Main channel in transformer diagram, tracking the cumulative sum giving the embedding vector.

## Articles/Resources

- Arena tutorials: https://arena3-chapter1-transformer-interp.streamlit.app/[1.2]_Intro_to_Mech_Interp

- Neel Nanda Quickstart Guide
- Neel Nanda Reading List